In [ ]:
import os, sys
import splink.comparison_library as cl
from splink.exploratory import completeness_chart
from splink import DuckDBAPI, Linker, SettingsCreator, block_on, splink_datasets,
from splink_tools import *
from pprint import pprint 
import pandas as pd
import numpy as np
import json
from pprint import pprint



ImportError: cannot import name 'Settings' from 'splink' (c:\Users\dec2g\anaconda3\envs\dan-env\lib\site-packages\splink\__init__.py)

In [2]:
def move_working_dir_to_repo_root(repo_name="orgsync"):
    """
    Move the current working directory to the root of the repository.
    """
    current_dir = os.getcwd()
    while os.path.basename(current_dir).lower() != repo_name:
        current_dir = os.path.dirname(current_dir)
    os.chdir(current_dir)
    print("Current working directory: ", os.getcwd())

move_working_dir_to_repo_root(repo_name="orgsync")

USE_TEST_DATA = False
# base paths 
test_dir = os.path.join("data", "test", "processed")
data_dir = os.path.join("data", "raw", "processed")

# load data as json
if not USE_TEST_DATA:
    with open(os.path.join(data_dir, "gtr_data.json"), "r") as f:
        gtr_json = json.load(f)
    with open(os.path.join(data_dir, "cordis_data.json"), "r") as f:
        cordis_json = json.load(f)

else:
    with open(os.path.join(test_dir, "gtr_data.json"), "r") as f:
        gtr_json = json.load(f)

    with open(os.path.join(test_dir, "cordis_data.json"), "r") as f:
        cordis_json = json.load(f)

pprint(gtr_json[0])
pprint(cordis_json[0])

Current working directory:  c:\Users\dec2g\GitHub\OrgSync
{'address.postCode': 'NE1 8QH',
 'address.region': 'North East',
 'address.type': 'MAIN_ADDRESS',
 'id': '5331B126-3AB4-4412-B56D-00E8F2796556',
 'link.EMPLOYEE': ['0CB4A538-AC14-4394-A10F-C9F955033EF3'],
 'name': 'NEWCASTLE CITY COUNCIL'}
{'city': 'MOOR ROW',
 'geolocation': '',
 'name': 'NUCLEAR DECOMMISSIONING AUTHORITY - NDA',
 'nutsCode': '',
 'organisationID': 999565019,
 'postCode': 'CA24 3HU',
 'rcn': 1906596,
 'shortName': 'NDA',
 'street': 'Westlakes Science PArk - Herdus House'}


In [3]:
# create df from json with specifications indicating the ordering of columns by name, removal of columns, and mappings for changing the name of columns  
# gtr

gtr_df = pd.DataFrame(gtr_json)
gtr_df.head()

map_names_gtr = {
    "id": "unique_id",
    "address.postCode": "postcode"

}
drop_cols_gtr = ["link.EMPLOYEE","address.type","address.region"]
gtr_order = ["unique_id", "name"]


gtr_df = drop_cols(gtr_df, drop_cols_gtr)
gtr_df = change_col_names(gtr_df, map_names_gtr)
gtr_df = reorder_cols(gtr_df, gtr_order)
gtr_df = clean_string_columns(gtr_df, columns=["name", "postcode"])
# gtr_df.head(50)


In [4]:
cordis_df = pd.DataFrame(cordis_json)
cordis_df.head()

map_names_cordis = {
    # "organisationID": "unique_id",
    "rcn": "unique_id",
    "postCode": "postcode", 
}

drop_cols_cordis = ["geolocation", "nutsCode", "shortName", "street", "city", "organisationID"]

cordis_order = ["unique_id", "name", "postcode"]

cordis_df = drop_cols(cordis_df, drop_cols_cordis)
cordis_df = change_col_names(cordis_df, map_names_cordis)
cordis_df = reorder_cols(cordis_df, cordis_order)
# print all enties in cordis_df["postcode"] that are not strings
# print(cordis_df[~cordis_df["postcode"].apply(lambda x: isinstance(x, str))])
# remove all entries in cordis_df["postcode"] that are not strings
cordis_df = cordis_df[cordis_df["postcode"].apply(lambda x: isinstance(x, str))]
cordis_df = blank_to_nan(cordis_df, column=None)
cordis_df = clean_string_columns(cordis_df, columns=["name", "postcode"])
# cordis_df.head(50)

In [5]:
# combine the cordis and gtr dataframe
df = pd.concat([cordis_df, gtr_df], ignore_index=True)
df.head(50)

,unique_id,name,postcode
0,1906596,nuclear decommissioning authority nda,ca24 3hu
1,2123954,south west tourism limited,ex2 5wt
2,1915686,welsh government,cf10 3nq
3,2128407,terrasalus limited,le15 9el
4,2168344,ol pharma partners ltd,sg4 7dp
5,1917645,forsite diagnostics ltd,yo41 1lz
6,1907389,st georges hospital medical school,sw17 0re
7,1909146,genetic alliance uk ltd,n1 3qp
8,2102071,fetal medicine foundation,w1g 6bg
9,2142093,centre of the cell,e1 2ab


In [11]:
### Remove columns with very low link

db_api = DuckDBAPI()
# Conver empty lists to null before running...#
completeness_chart(df, db_api=db_api, table_names_for_chart=["df"])




alt.LayerChart(...)

In [24]:
from splink.blocking_analysis import count_comparisons_from_blocking_rule


br = block_on("substr(name, 1,1)", "name")

counts = count_comparisons_from_blocking_rule(
    table_or_tables=df,
    blocking_rule=br,
    link_type="dedupe_only",
    db_api=db_api,
)

counts

{'number_of_comparisons_generated_pre_filter_conditions': 14162183,
 'number_of_comparisons_to_be_scored_post_filter_conditions': 69331,
 'filter_conditions_identified': '',
 'equi_join_conditions_identified': 'SUBSTRING(l.name, 1, 1) = SUBSTRING(r.name, 1, 1) AND l."name" = r."name"',
 'link_type_join_condition': 'where l."unique_id" < r."unique_id"'}

Comparisons

https://moj-analytical-services.github.io/splink/demos/tutorials/04_Estimating_model_parameters.html

https://moj-analytical-services.github.io/splink/topic_guides/comparisons/out_of_the_box_comparisons.html

https://moj-analytical-services.github.io/splink/api_docs/api_docs_index.html

Can we block on fuzzy match above some threshold?

If we can map to one standardised column (even if sparse), 
```python
linker.estimate_m_from_label_column("standardised_column")
```

Ideally, we might want to try 1=1 exluding the id field. 

Fuzzy blocking rules
https://moj-analytical-services.github.io/splink/topic_guides/blocking/blocking_rules.html#:~:text=It%20is%20possible%20to%20use%20more%20complex%20blocking,l.first_name%20and%20r.first_name%20and%20levenshtein%28l.surname%2C%20r.surname%29%20%3C%203

In [ ]:
settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
        cl.NameComparison("name"),
        cl.PostcodeComparison("postcode"),
        # cl.ExactMatch("postcode").configure(
        #     term_frequency_adjustments=True
        # )
    ],
    blocking_rules_to_generate_predictions=[
        block_on("name"),
        block_on("postcode")
        # "1=1"
        # "l.name and r.name" #and levenshtein(l.name, r.name) < 4"
        # block_on("name"),
        # block_on("postcode")
    ],
    retain_intermediate_calculation_columns=True
)

deterministic_rules = [
    block_on("name"),
    block_on("postcode")
    # "1=1"
    # "l.name and r.name"# and levenshtein(l.name, r.name) < 2"
]


linker = Linker(df, settings, db_api=DuckDBAPI())

linker.training.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)

linker.training.estimate_u_using_random_sampling(max_pairs=1e6)

training_blocking_rule = block_on("postcode")
    # "l.name and r.name" #and levenshtein(l.name, r.name) < 4"


training_session_fname_sname = (
    linker.training.estimate_parameters_using_expectation_maximisation(training_blocking_rule)
)



Probability two random records match is estimated to be  0.00635.
This means that amongst all possible pairwise record comparisons, one in 157.45 are expected to match.  With 5,572,776,378 total possible comparisons, we expect a total of around 35,393,081.43 matching pairs
You are using the default value for `max_pairs`, which may be too small and thus lead to inaccurate estimates for your model's u-parameters. Consider increasing to 1e8 or 1e9, which will result in more accurate estimates, but with a longer run time.
----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - name (no m values are trained).
    - postcode (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."postcode" = r."postcode"

Parameter estimates will be made for the following comparison(s):
    - name

Parameter estim

In [ ]:
settings = linker.misc.save_model_to_json(
    "../demo_settings/saved_model_from_demo.json", overwrite=True
)
linker.evaluation.unlinkables_chart()